In [0]:
spark.read.format('parquet').options(inferSchema=True, header=True).load('/Volumes/true_catalog/bronze/kagle_dataset_raw/nyc-rideshare-raw-data/rideshare_data.parquet').limit(10).display()

In [0]:
spark.read.format('csv').options(inferSchema=True, header=True).load('/Volumes/true_catalog/bronze/kagle_dataset_raw/nyc-rideshare-raw-data/taxi_zone_lookup.csv').limit(10).display()